In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

import keras
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
from keras.applications import MobileNet
from keras.applications import ResNet50
from keras.applications import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
# from keras.applications.inception_resnet_v2 import preprocess_input
# from keras.applications.resnet50 import preprocess_input
# from keras.applications.mobilenet import preprocess_input
from keras.utils.training_utils import multi_gpu_model
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.preprocessing.image import load_img
from keras import Model
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import concatenate
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute, Input
from keras.layers import UpSampling2D, Conv2D, Conv2DTranspose, Concatenate, MaxPooling2D
from keras.layers import concatenate, Dropout, BatchNormalization, Lambda, Flatten, Activation
from keras.losses import binary_crossentropy

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Conv1D, LSTM, Dense, Dropout, Bidirectional
#if len(get_available_gpus())>0:
    # https://twitter.com/fchollet/status/918170264608817152?lang=en
from keras.layers import CuDNNLSTM as LSTM # this one is about 3x faster on GPU instances
from keras import backend as K
from keras.utils.training_utils import multi_gpu_model
from alt_model_checkpoint import AltModelCheckpoint
import tensorflow as tf


def step_decay_exp(epoch):
    initial_lrate = 0.01
    drop = 0.5 # drop by how much? 0.5 = half each time
    epochs_drop = 10.0 # drop how many x epochs
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

def cse_block(input, prefix, ratio=16):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D(name = prefix + "_cse_gap1")(init)
    se = Reshape(se_shape, name = prefix + "_cse_reshape1")(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False,
              name = prefix + "_cse_fc1")(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False,
              name = prefix + "_cse_fc2")(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se], name = prefix + "_cse_mul1")
    return x

def sse_block(input, prefix):
    init = input
    conv = Conv2D(1, (1,1), padding = 'same', kernel_initializer = 'he_normal',
                  activation = 'sigmoid', strides = (1,1), name = prefix + "_sse_conv")(input)
    conv = Multiply(name = prefix + "_sse_mul1")([input, conv])
    return conv

def csse_block(x, prefix):
    cse = cse_block(x, prefix)
    sse = sse_block(x, prefix)
    out = Add(name = prefix + "_csse_mul")([cse, sse])
    return out


def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)


def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)


def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
 #   stroke_vec = literal_eval(raw_strokes) # string->list
    # unwrap the list
    stroke_vec = ast.literal_eval(raw_strokes)
    in_strokes = [(xi,yi,zi,i)  
     for i,(x,y,z) in enumerate(stroke_vec) 
     for xi,yi,zi in zip(x,y,z)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
#    c_strokes[:,3] = [1]+np.diff(c_strokes[:,3]).tolist()
#    c_strokes[:,3] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)

### Image Generator
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            # can try to do exponential smoothing / decay
            # how about embedding different elements/colours to different channels
            # one channel has raw 0/255
            # one channel has decay 0->255
            # one channel has decay 0->255 but with different line width
            color = 255 - min(t, 15) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
#             print('Chosen {}'.format(k))
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['time'].apply(ast.literal_eval)
                x = np.zeros((len(df), size, size, 3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 1] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=False)
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 2] = draw_cv2(raw_strokes, size=size, lw=4,
                                             time_color=True)
#                 x = np.repeat(x, 3, axis = 3)
                x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                df['time'] = df['time'].map(_stack_it)
                x2 = np.stack(df['time'], 0)
                yield [x, x2], y

def df_to_image_array_xd(df, size, lw=6, time_color=True):
    df['drawing'] = df['time'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 3))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 1] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=False)
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 2] = draw_cv2(raw_strokes, size=size, lw=4, time_color=True)
    x = preprocess_input(x).astype(np.float32)
    df['time'] = df['time'].map(_stack_it)
    x2 = np.stack(df['time'], 0)
    return  [x, x2]

/home/tom/anaconda2/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
DP_DIR = './input/train_time_100k/'
INPUT_DIR = './input/'
STROKE_COUNT = 100
BASE_SIZE = 256
NCSVS = 20
NCATS = 340
np.random.seed(seed=123)
tf.set_random_seed(seed=123)
size = 128

In [20]:
base_model_cnn=None
with tf.device("/cpu:0"):
    # create the base pre-trained model
    base_model_cnn  = Xception(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')

   
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
   


    # add a global spatial average pooling layer
    x = base_model_cnn.output
    x = GlobalAveragePooling2D(name='avg_pool_n')(x)
    x=Dense(2048, activation = 'relu')(x) 
    # let's add a fully-connected layer
    x = Dropout(0.2)(x)
    predictions = Dense(340, activation='softmax', name='predictions')(x)
    base_model_cnn  = Model(inputs=base_model_cnn.input,  outputs=predictions)
  #  base_model_cnn.load_weights('keras_models/xception_model_tom_095433.h5')

    for layer in base_model_cnn.layers:
        layer.trainable = False
        
    base_model_cnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [21]:
base_model_rnn=None
with tf.device("cpu:0"):
    
    base_model_rnn = Sequential()
    base_model_rnn.add(BatchNormalization(input_shape = (None,)+(4,)))
    # filter count and length are taken from the script https://github.com/tensorflow/models/blob/master/tutorials/rnn/quickdraw/train_model.py
    base_model_rnn.add(Conv1D(256, (5,), activation = 'relu'))
    base_model_rnn.add(Dropout(0.2))
    #base_rnn_model.add(Conv1D(512, (5,), activation = 'relu'))
    #base_rnn_model.add(Dropout(0.2))
    #base_rnn_model.add(Conv1D(512, (3,), activation = 'relu'))
    #base_rnn_model.add(Dropout(0.2))
    #base_rnn_model.add(Bidirectional(LSTM(256, return_sequences = True)))
    #base_rnn_model.add(Dropout(0.2))
    base_model_rnn.add(Bidirectional(LSTM(512, return_sequences = True)))
    base_model_rnn.add(Dropout(0.2))
    base_model_rnn.add(Bidirectional(LSTM(512, return_sequences = False)))
    base_model_rnn.add(Dense(2048, activation = 'relu'))
    base_model_rnn.add(Dropout(0.2))
    base_model_rnn.add(Dense(NCATS, activation = 'softmax'))
   # base_model_rnn.load_weights('keras_models/stroke_lstm_bidirectional_relu_weights_0926.best.hdf5')
    for layer in base_model_rnn.layers:
        layer.trainable = False
        
    base_model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_10 (Batc (None, None, 4)           16        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 256)         5376      
_________________________________________________________________
dropout_9 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 1024)        3153920   
_________________________________________________________________
dropout_10 (Dropout)         (None, None, 1024)        0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1024)              6299648   
_________________________________________________________________
dense_8 (Dense)              (None, 2048)              2099200   
__________

In [22]:
with tf.device("cpu:0"):
    cnn_output=base_model_cnn.layers[-3].output
    rnn_output=base_model_rnn.layers[-3].output
    
    cnn_input=base_model_cnn.input
    rnn_input=base_model_rnn.input
    
    x = concatenate([cnn_output, rnn_output])
    x = Dropout(0.3)(x)
    x = Dense(NCATS, activation='softmax')(x)
    base_model = Model([cnn_input, rnn_input], x)
    base_model.load_weights('./keras_models/xception_rnn_model_tom_last_0954.h5')
    
    for layer in base_model_cnn.layers:
        layer.trainable = True
    for layer in base_model_rnn.layers:
        layer.trainable = True
    base_model.summary()
gpu_model = multi_gpu_model(base_model, gpus=4)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [23]:
### Xception
from keras.callbacks import CSVLogger

EPOCHS = 8
TOTAL_TRAIN = 2250000
BATCH_SIZE = 128 * 4
STEPS = TOTAL_TRAIN // BATCH_SIZE

train_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=range(NCSVS - 1))
valid_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=NCSVS - 1)

def step_decay_exp(epoch):
    initial_lrate = 0.001
    drop = 0.5 # drop by how much? 0.5 = half each time
    epochs_drop = 7.0 # drop how many x epochs
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return max(1e-6, lrate)

callbacks = [
    AltModelCheckpoint('xception_rnn_model_tom.h5',base_model, monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = True, verbose = 1),
    AltModelCheckpoint('xception_rnn_model_tom_last.h5', base_model,monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = False, verbose = 0),
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.66, patience=3,
                      min_lr = 0.000001, 
                      mode='max', 
                      cooldown=3, verbose=1),
#     LearningRateScheduler(step_decay_exp),
    CSVLogger('xception_log.csv', append=True, separator=';')
]

# sgdOptimiser = SGD(lr = 0.001, momentum=0.9, decay = 0.0001,  nesterov=True)

gpu_model.compile(optimizer=Adam(lr=0.0005), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

gpu_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data = valid_datagen, validation_steps = STEPS // 13, # can try STEPS // 6 or 7 for more consistent validation
    callbacks = callbacks
)

Epoch 1/8
4394/4394 [==============================] - 5703s 1s/step - loss: 0.5645 - categorical_crossentropy: 0.5645 - categorical_accuracy: 0.8485 - top_3_accuracy: 0.9501 - val_loss: 0.5324 - val_categorical_crossentropy: 0.5324 - val_categorical_accuracy: 0.8566 - val_top_3_accuracy: 0.9539

Epoch 00001: val_top_3_accuracy improved from -inf to 0.95392, saving model to xception_rnn_model_tom.h5
Epoch 2/8
4394/4394 [==============================] - 5616s 1s/step - loss: 0.5575 - categorical_crossentropy: 0.5575 - categorical_accuracy: 0.8505 - top_3_accuracy: 0.9508 - val_loss: 0.5369 - val_categorical_crossentropy: 0.5369 - val_categorical_accuracy: 0.8561 - val_top_3_accuracy: 0.9535

Epoch 00002: val_top_3_accuracy did not improve from 0.95392
Epoch 3/8
4394/4394 [==============================] - 5646s 1s/step - loss: 0.5545 - categorical_crossentropy: 0.5545 - categorical_accuracy: 0.8513 - top_3_accuracy: 0.9513 - val_loss: 0.5411 - val_categorical_crossentropy: 0.5411 - val

In [ ]:


model = multi_gpu_model(base_model, gpus=4)       
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00005), 
              metrics=[top_k_categorical_accuracy, 'accuracy'])


gpu_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=10, verbose=1,
    validation_data = valid_datagen, validation_steps = STEPS // 13, # can try STEPS // 6 or 7 for more consistent validation
    callbacks = callbacks
)

In [24]:
# run the validation
import gc
#gpu_model.load_weights('xception_model.h5') # best run - otherwise choose xception_model_last.h5
#gpu_model.load_weights('xception_model_last.h5')
valid_batch_size = 30000
valid_data = pd.read_csv(os.path.join('./input/train_simplified_time_split_20/train_k19.csv.gz'))

num_batch = np.ceil(len(valid_data)/valid_batch_size) # can override to reduce the number of validation
counter = 0
nrows = 0
pred_valid_array = []

while True:
    batch_files = valid_data[valid_batch_size*counter:valid_batch_size*(counter+1)]
    
    x_valid = df_to_image_array_xd(batch_files, size)
    p_valid = gpu_model.predict(x_valid, batch_size=1024, verbose=1)
    
    if counter == 0:
        pred_valid_array = p_valid
    else:
        pred_valid_array = np.concatenate((pred_valid_array, p_valid), axis = 0)
    print('Pred shape: {}'.format(pred_valid_array.shape))
    
    x_valid = []
    gc.collect()
    
    counter += 1
    nrows += len(batch_files)
    if counter >= num_batch:
            break

print(nrows)
map3 = mapk(valid_data[['y']].values, preds2catids(pred_valid_array).values)
print('Map3: {:.3f}'.format(map3))

/home/tom/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tom/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


30000/30000 [==============================] - 38s 1ms/step
Pred shape: (30000, 340)


0

30000/30000 [==============================] - 24s 784us/step
Pred shape: (60000, 340)


0

...

30000/30000 [==============================] - 26s 851us/step
Pred shape: (2460000, 340)


0

26554/26554 [==============================] - 24s 888us/step
Pred shape: (2486554, 340)


0

2486554
Map3: 0.896


In [25]:
test_batch_size = 10000
test = pd.read_csv(os.path.join('./input/test_simplified_time.csv'))

num_batch = np.ceil(len(test)/test_batch_size)
counter = 0
nrows = 0
pred_array = []

while True:
    batch_files = test[test_batch_size*counter:test_batch_size*(counter+1)]
    
    x_test = df_to_image_array_xd(batch_files, size)
    p_test = gpu_model.predict(x_test, batch_size=256, verbose=1)
    
    if counter == 0:
        pred_array = p_test
    else:
        pred_array = np.concatenate((pred_array, p_test), axis = 0)
    print('Pred shape: {}'.format(pred_array.shape))
    
    x_valid = []
    gc.collect()
    
    counter += 1
    nrows += len(batch_files)
    if counter >= num_batch:
            break

print(nrows)

/home/tom/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tom/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10000/10000 [==============================] - 10s 1ms/step
Pred shape: (10000, 340)


0

10000/10000 [==============================] - 11s 1ms/step
Pred shape: (20000, 340)


0

...

10000/10000 [==============================] - 10s 1ms/step
Pred shape: (110000, 340)


0

2199/2199 [==============================] - 2s 1ms/step
Pred shape: (112199, 340)


0

112199


In [26]:
# standard code from kernel to convert to submission file
top3 = preds2catids(pred_array)
top3.head()
top3.shape

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

,a,b,c
0,234,281,285
1,144,36,226
2,305,62,110
3,187,111,304
4,56,113,112


(112199, 3)

,a,b,c
0,radio,stereo,stove
1,hockey_puck,bottlecap,pool
2,The_Great_Wall_of_China,castle,fence
3,mountain,finger,The_Eiffel_Tower
4,campfire,fireplace,fire_hydrant


(112199, 3)

In [27]:
### This section is for ensemble output only
def preds2catids_pr(predictions, k=5):
    topk = pd.DataFrame(np.argsort(-predictions, axis=1)[:, :k], columns=[chr(ord('a')+x) for x in range(k)])
    
    cats = list_all_categories()
    id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
    topcats = topk.replace(id2cat)
    
    p_pr = pd.DataFrame(-np.sort(-predictions, axis=1)[:, :k])
    p_all = pd.concat([topcats, p_pr], axis=1)
    
    # construct the word
    p_all['word'] = ""
    
    for l in range(k):
        p_all['word'] += (p_all[chr(ord('a')+l)] + " " + p_all[l].astype(str) + " ")
    
    # reuse the test CSV for ids
    test_df = pd.read_csv(os.path.join('./input/test_simplified.csv'))
    test_df['word'] = p_all['word']
    
    return test_df 

x = preds2catids_pr(pred_array)
x.head()

# generate the top 5 pr CSV
x[['key_id', 'word']].to_csv('xception128_rnn_top5_last.csv', index = False)

,key_id,countrycode,drawing,word
0,9000003627287624,DE,"[[[17, 18, 20, 25, 137, 174, 242, 249, 251, 25...",radio 0.8601467 stereo 0.13984916 stove 2.2499...
1,9000010688666847,UA,"[[[174, 145, 106, 38, 11, 4, 4, 15, 29, 78, 16...",hockey_puck 0.9357352 bottlecap 0.043401606 po...
2,9000023642890129,BG,"[[[0, 12, 14, 17, 16, 24, 55, 57, 60, 79, 82, ...",The_Great_Wall_of_China 0.933876 castle 0.0563...
3,9000038588854897,US,"[[[0, 9, 23, 40, 54, 60, 81, 105, 123, 167, 20...",mountain 0.9999802 finger 5.4075167e-06 The_Ei...
4,9000052667981386,AR,"[[[87, 82, 71, 63, 66, 92, 96, 95], [220, 218,...",campfire 0.56525826 fireplace 0.4194067 fire_h...


In [28]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv('xception128_rnn_map3_{}_v1.csv'.format(int(map3 * 10**4)), index=False)
submission.head()
submission.shape

,key_id,word
0,9000003627287624,radio stereo stove
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain finger The_Eiffel_Tower
4,9000052667981386,campfire fireplace fire_hydrant


(112199, 2)

# Xception 256

In [29]:
base_model = Xception(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(340, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = x)
gpu_model = multi_gpu_model(model, 2)
print(gpu_model.summary())
gpu_model.load_weights('xception_model256_cur.h5')

KeyboardInterrupt: 

In [ ]:
### Xception 256
from keras.callbacks import CSVLogger

EPOCHS = 100
TOTAL_TRAIN = 2250000
BATCH_SIZE = 32 * 2
STEPS = TOTAL_TRAIN // BATCH_SIZE

train_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=range(NCSVS - 1))
valid_datagen = image_generator_xd(size=size, batchsize=BATCH_SIZE, ks=NCSVS - 1)

def step_decay_exp(epoch):
    initial_lrate = 0.00075
    drop = 0.5 # drop by how much? 0.5 = half each time
    epochs_drop = 7.0 # drop how many x epochs
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return max(1e-6, lrate)

callbacks = [
    ModelCheckpoint('xception_model256.h5', monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = True, verbose = 1),
    ModelCheckpoint('xception_model256_cur.h5', monitor = 'val_top_3_accuracy', mode = 'max', save_best_only = False, verbose = 0),
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.66, patience=3,
                      min_lr = 0.000001, 
#                       min_delta=0.005, 
                      mode='max', 
                      cooldown=3, verbose=1),
#     LearningRateScheduler(step_decay_exp),
    CSVLogger('xception_log256.csv', append=True, separator=';')
]

# sgdOptimiser = SGD(lr = 0.001,  momentum=0.9, decay = 0.0001,  nesterov=True)

gpu_model.compile(optimizer=Adam(lr=0.00015), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

gpu_model.load_weights('xception_model256_cur.h5')

gpu_model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data = valid_datagen, validation_steps = STEPS // 8,
    callbacks = callbacks
)


In [ ]:
test_batch_size = 5000
test = pd.read_csv(os.path.join('test_simplified.csv'))

num_batch = np.ceil(len(test)/test_batch_size)
counter = 0
nrows = 0
pred_array = []

while True:
    batch_files = test[test_batch_size*counter:test_batch_size*(counter+1)]
    
    x_test = df_to_image_array_xd(batch_files, 256)
    p_test = gpu_model.predict(x_test, batch_size=256, verbose=1)
    
    if counter == 0:
        pred_array = p_test
    else:
        pred_array = np.concatenate((pred_array, p_test), axis = 0)
    print('Pred shape: {}'.format(pred_array.shape))
    
    x_valid = []
    gc.collect()
    
    counter += 1
    nrows += len(batch_files)
    if counter >= num_batch:
            break

print(nrows)

In [ ]:
x = preds2catids_pr(pred_array)
x.head()

# generate the top 5 pr CSV
x[['key_id', 'word']].to_csv('xception256_top5_last.csv', index = False)